# 2개 카테고리 /4개 카테고리에 따른 각각의 이미지 segmentation 시각화 과정

# 주차 공간 인식을 위한 Mask R-CNN 모델

## 필요한 라이브러리 임포트

먼저 필요한 라이브러리들을 임포트합니다.


In [ ]:
import os
import torch
import torchvision
import cv2
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt


## GPU 설정

이 노트북은 GPU를 사용하여 모델을 학습 및 평가합니다. GPU가 사용 가능한 경우, GPU를 사용하도록 설정합니다.


In [ ]:
# GPU 사용 여부 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


## 모델 로드 함수

이 함수는 사전 학습된 Mask R-CNN 모델을 로드하고, 주차 공간, 도로, 차량, 사람의 클래스를 포함한 커스텀 모델로 수정합니다.

### 변수:
- `num_classes`: 모델에서 예측할 클래스의 수입니다.
- `weights_path`: 모델 가중치 파일의 경로입니다.


In [ ]:
def load_model(num_classes, weights_path):
    model = maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        model.roi_heads.mask_predictor.conv5_mask.in_channels, 256, num_classes)
    model.load_state_dict(torch.load(weights_path))
    model.to(device)
    model.eval()
    return model


## 이미지 전처리 함수

이 함수는 이미지를 Tensor 형식으로 변환하여 모델에 입력할 수 있는 형태로 만들어줍니다.

### 변수:
- `train`: 학습 모드 여부를 나타냅니다. 여기서는 평가 모드로 사용됩니다.


In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(transforms)


## 예측 시각화 함수

이 함수는 주어진 이미지에 대해 모델의 예측을 시각화하고, 주차 공간, 차량, 사람의 개수를 화면에 표시합니다.

### 인자:
- `model`: 학습된 Mask R-CNN 모델입니다.
- `img_path`: 예측할 이미지의 경로입니다.
- `device`: GPU 또는 CPU 장치를 나타냅니다.
- `num_classes`: 모델에서 사용하는 클래스의 수입니다.


In [ ]:
def visualize_predictions(model, img_path, device, num_classes):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    transform = get_transform(train=False)
    img_tensor = transform(img).to(device)
    with torch.no_grad():
        predictions = model([img_tensor])

    img_np = np.array(img)

    # 카테고리 이름 및 색상 정의 (1: 주차 공간, 2: 도로, 3: 사람, 4: 차량)
    category_names = {1: "Parking Space", 2: "Driveable Space", 3: "Person", 4: "Vehicle"}
    colors = {
        1: (0, 255, 0),  # Parking Space: Green
        2: (255, 0, 0),  # Driveable Space: Red
        3: (255, 255, 0),  # Person: Yellow
        4: (0, 0, 255)  # Vehicle: Blue
    }

    parking_space_count = 0
    vehicle_count = 0
    person_count = 0

    for i in range(len(predictions[0]['masks'])):
        score = predictions[0]['scores'][i].cpu().item()
        label = predictions[0]['labels'][i].cpu().item()

        if score > 0.6:  # Threshold
            if label == 1:  # 주차 공간
                parking_space_count += 1
            elif label == 4 and num_classes > 3:  # 차량
                vehicle_count += 1
            elif label == 3 and num_classes > 3:  # 사람
                person_count += 1

            color = colors.get(label, (255, 255, 255))  # 카테고리 색상을 가져옴. 없으면 흰색으로 설정.

            # Mask를 Segmentation 결과에 반영
            mask = predictions[0]['masks'][i, 0].cpu().numpy()
            mask = mask > 0.3
            color_mask = np.zeros_like(img_np)
            color_mask[mask] = color

            img_np = cv2.addWeighted(img_np, 1, color_mask, 0.5, 0)

    # 텍스트 가독성을 높이기 위한 반투명 박스 추가
    box_coords = [(5, 5), (350, 120)]
    cv2.rectangle(img_np, box_coords[0], box_coords[1], (0, 0, 0), cv2.FILLED)
    alpha = 0.4  # 투명도 조절 (0: 투명, 1: 불투명)
    img_np = cv2.addWeighted(img_np, alpha, img_np, 1 - alpha, 0, img_np)

    # 텍스트 추가
    cv2.putText(img_np, f"Parking Spaces: {parking_space_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    if num_classes > 3:
        cv2.putText(img_np, f"Vehicles: {vehicle_count}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(img_np, f"Persons: {person_count}", (10, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(img_np)
    plt.axis("off")
    plt.show()


## 모델 로드 및 예측 시각화 실행

이제 모델을 로드하고, 예측을 시각화해보겠습니다. `num_classes`를 변경하여 필요한 클래스 수를 설정할 수 있습니다. 2로 하면(주차공간,도로) 4로하면 (기존 카테고리 + 사람,차량)


In [ ]:
# 이미지 경로와 가중치 경로 설정
img_path = "path_to_your_image.jpg"
weights_path = "path_to_your_weights.pth"

# 모델의 출력층을 커스터마이즈 (background + 4 classes)
num_classes = 3  # background + 2 classes (Parking Space, Driveable Space)
# num_classes = 5  # background + 4 classes (Parking Space, Driveable Space, Person, Vehicle)

# 모델 로드
model = load_model(num_classes, weights_path)

# 예측 시각화 실행
visualize_predictions(model, img_path, device, num_classes)


# 동영상 추론 과정 을 위한 데이터 전처리 



동영상의 배속을 0.3으로 만들어서 segment의 변동성을 줄여주기 위해서 진행했습니다.
이 과정을 진행 안할시에 너무 빨리 주차공간을 카운트하기 때문에 관찰이 힘들어서 이와같이 진행했습니다.

In [ ]:
from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx
# 원본 동영상 파일 경로
input_file = r"C:\088.주차 공간 탐색을 위한 차량 관점 복합 데이터\market.mp4"
# 변환된 동영상 파일 경로
output_file = r"C:\088.주차 공간 탐색을 위한 차량 관점 복합 데이터\market_0.3.mp4"

# 동영상 파일 로드
clip = VideoFileClip(input_file)

# 0.3배속으로 동영상 속도 변환
slow_clip = clip.fx(vfx.speedx, 0.3)

# 변환된 동영상을 파일로 저장
slow_clip.write_videofile(output_file, codec="libx264", audio_codec="aac")

# **Mask R-CNN을 이용한 주차 공간 및 도로 인식**
Mask R-CNN을 이용하여 이미지 및 비디오에서 주차 공간과 도로를 인식하고, 주차 공간의 수를 계산하는 방법을 구현합니다.

---

## **1. 라이브러리 임포트 및 장치 설정**

먼저, 필요한 라이브러리들을 임포트하고, GPU 사용을 설정합니다.


In [ ]:
# 라이브러리 임포트
import os
import torch
import torchvision
import cv2
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from PIL import Image
from tqdm import tqdm

# 장치 설정 (CUDA 사용 가능 시 GPU, 그렇지 않으면 CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


---

## **2. Mask R-CNN 모델 불러오기 및 수정**

이 함수는 사전 학습된 Mask R-CNN 모델을 불러오고, 사용자 정의 클래스로 수정합니다. 주차 공간 및 도로 클래스를 포함하여 모델을 학습시키기 위해 이 부분을 수정할 수 있습니다.
    사전 학습된 Mask R-CNN 모델을 불러오고, 사용자 정의 클래스로 수정합니다.

    Args:
    - num_classes (int): 클래스의 수 (배경 포함).
    - weights_path (str): 모델 가중치가 저장된 경로.

    Returns:
    - model (torch.nn.Module): 불러온 Mask R-CNN 모델.

In [ ]:
def load_model(num_classes, weights_path):

    # 사전 학습된 Mask R-CNN 모델 로드
    model = maskrcnn_resnet50_fpn(weights="DEFAULT")
    
    # 클래스 예측기를 사용자 정의 클래스 수에 맞게 수정
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    # 마스크 예측기를 사용자 정의 클래스 수에 맞게 수정
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        model.roi_heads.mask_predictor.conv5_mask.in_channels, 256, num_classes)
    
    # 저장된 가중치 로드
    model.load_state_dict(torch.load(weights_path))
    
    # 모델을 GPU 또는 CPU로 이동
    model.to(device)
    
    # 평가 모드로 전환
    model.eval()
    
    return model


---

## **3. Intersection over Union (IoU) 계산**

IoU는 두 개의 바운딩 박스 간의 겹치는 부분을 계산하여 얼마나 겹치는지를 나타내는 지표입니다. 이 함수는 IoU를 계산합니다.
    두 개의 바운딩 박스 간의 IoU(Intersection over Union)를 계산합니다.

    Args:
    - box1 (list): 첫 번째 바운딩 박스.
    - box2 (list): 두 번째 바운딩 박스.

    Returns:
    - iou (float): 계산된 IoU 값.


In [ ]:
def calculate_iou(box1, box2):

    # 두 박스 간의 교집합 영역 계산
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # 교집합 면적 계산
    inter_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    
    # 각 박스의 면적 계산
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # IoU 계산
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou


---

## **4. IoU를 기반으로 겹치는 박스 필터링**

여러 바운딩 박스가 있을 때, 겹치는 박스를 제거하여 하나의 박스만 남기는 방법을 사용합니다. 이 함수는 IoU 임계값을 기준으로 겹치는 박스를 필터링합니다.
 
    IoU 임계값을 기준으로 겹치는 바운딩 박스를 필터링합니다.

    Args:
    - boxes (list): 바운딩 박스 리스트.
    - iou_threshold (float): IoU 임계값.

    Returns:
    - filtered_boxes (list): 필터링된 바운딩 박스 리스트.
    


In [ ]:
def filter_overlapping_boxes(boxes, iou_threshold=0.5):

    filtered_boxes = []
    for i, box in enumerate(boxes):
        overlap = False
        for j, kept_box in enumerate(filtered_boxes):
            # 두 박스 간의 IoU가 임계값을 넘는 경우 겹침으로 판단
            if calculate_iou(box, kept_box) > iou_threshold:
                overlap = True
                break
        # 겹치지 않는 박스만 필터링된 리스트에 추가
        if not overlap:
            filtered_boxes.append(box)
    return filtered_boxes


---

## **5. 각 프레임에서 예측 시각화**

이 함수는 주어진 프레임에 대해 Mask R-CNN 모델을 사용하여 예측을 수행하고, 주차 공간의 수를 계산하며, 시각화된 결과를 반환합니다.

    각 프레임에서 예측 결과를 시각화하고 주차 공간의 수를 계산합니다.

    Args:
    - model (torch.nn.Module): Mask R-CNN 모델.
    - frame (np.array): 비디오 프레임.

    Returns:
    - img_np (np.array): 예측이 시각화된 프레임.


In [ ]:
def visualize_predictions(model, frame):

    transform = torchvision.transforms.ToTensor()
    img_tensor = transform(frame).to(device).unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
        predictions = model(img_tensor)

    img_np = np.array(frame)
    category_names = {1: "Parking Space", 2: "Driveable Space"}  # 카테고리 이름 정의

    parking_boxes = []
    for i in range(len(predictions[0]['masks'])):
        score = predictions[0]['scores'][i].cpu().item()
        label = predictions[0]['labels'][i].cpu().item()

        if score > 0.8:  # 신뢰도 임계값 확인
            box = predictions[0]['boxes'][i].cpu().numpy().astype(int)
            mask = predictions[0]['masks'][i, 0].cpu().numpy()
            mask = mask > 0.3  # 마스크 임계값 적용
            color = np.zeros_like(img_np)

            if label == 1:  # 주차 공간인 경우
                color[mask] = [0, 255, 0]  # 초록색으로 표시
                parking_boxes.append(box)
            elif label == 2:  # 도로 공간인 경우
                color[mask] = [255, 0, 0]  # 파란색으로 표시

            img_np = cv2.addWeighted(img_np, 1, color, 0.5, 0)
            
            # 객체의 신뢰도 출력
            text = f"{category_names[label]}: {score * 100:.2f}%"
            cv2.putText(img_np, text, (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # 겹치는 주차 공간 박스를 필터링
    filtered_parking_boxes = filter_overlapping_boxes(parking_boxes)

    # 주차 공간 수를 화면에 표시
    parking_spaces = len(filtered_parking_boxes)
    cv2.putText(img_np, f"Parking Spaces: {parking_spaces}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return img_np


---

## **6. 비디오 처리**

이 함수는 비디오 파일을 프레임 단위로 처리하여, 각 프레임에서 주차 공간을 계산하고 예측 결과를 시각화한 후, 결과 비디오를 저장합니다.


In [ ]:
def process_video(video_path, model, output_path, playback_speed=0.3):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for _ in tqdm(range(total_frames), desc="Processing video"):
        ret, frame = cap.read()
        if not ret:
            break

        result_frame = visualize_predictions(model, frame)

        # Adjust FPS to match the playback speed
        adjusted_fps = int(fps * playback_speed)

        # Display adjusted FPS on the frame
        cv2.putText(result_frame, f"FPS: {adjusted_fps}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        out.write(result_frame)

    cap.release()
    out.release()

---

## **7. 메인 실행 코드**

이 부분은 노트북의 메인 코드입니다. 모델을 불러오고, 비디오 파일을 처리하여 결과를 저장합니다.


In [ ]:
# 파라미터 설정
num_classes = 3  # 배경 + 2개의 클래스 (주차 공간, 도로)
weights_path = "model_weights/best_model0828.pth"  # 저장된 모델 가중치 경로
video_path = "C:/088.주차 공간 탐색을 위한 차량 관점 복합 데이터/market_0.3.mp4"  # 입력 비디오 파일 경로
output_path = "C:/088.주차 공간 탐색을 위한 차량 관점 복합 데이터/processed_output_market2_r.mp4"  # 출력 비디오 파일 경로
playback_speed = 0.3  # Video playback speed (0.3x slower)
# 모델 로드
model = load_model(num_classes, weights_path)

# 비디오 처리 및 결과 저장
process_video(video_path, model, output_path, playback_speed)

print(f"처리가 완료되었습니다. 결과는 {output_path}에 저장되었습니다.")
